In [1]:
import sys
import os

current_dir = os.path.abspath(os.getcwd())
project_home_dir = os.path.abspath(os.path.join(current_dir, os.pardir))

sys.path.append(project_home_dir)

In [2]:
import torch
from transformers import SegformerForSemanticSegmentation, SegformerFeatureExtractor
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import wandb

from src.data_preprocessing import prepare_dataset
from src.model import train_segformer

/home/inside-tech/Desktop/image_segmentation/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class_rgb_values = {
    0: [0, 0, 255],      # Class 0 is represented by blue pixels
    1: [0, 255, 0],      # Class 1 is represented by green pixels
    2: [255, 0, 0],      # Class 2 is represented by red pixels
    3: [0, 170, 255],    # Class 3 is represented by orange pixels
    4: [255, 85, 255]    # Class 4 is represented by pink pixels
}

In [ ]:
img_dir = "/home/inside-tech/Desktop/image_segmentation/data/raw/_4_classi/images"
mask_dir = "/home/inside-tech/Desktop/image_segmentation/data/raw/_4_classi/images"

wandb.init(project="CowSegmentation", name="SegFormer")

# Prepara dataloader
train_loader, val_loader, test_loader, full_dataset = prepare_dataset(
    img_dir=img_dir,
    mask_dir=mask_dir,
    class_rgb_values=class_rgb_values,
    batch_size=8
)

# Calcola i pesi delle classi per il bilanciamento
class_weights = full_dataset.get_class_weight()

# Esegui il training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model, trainer = train_segformer(
    train_loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    num_classes=len(class_rgb_values),
    epochs=50,
    learning_rate=1e-4,
    device=device,
    class_weights=class_weights
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: vincenzo-civale (vincenzo-civale-universi-degli-studi-di-firenze) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Dataset caricato: 646 immagini con maschere corrispondenti
Divisione dataset: 451 train, 65 validation, 130 test


Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/segformer-b0-finetuned-ade-512-512 and are newly initialized because the shapes did not match:
- decode_head.classifier.bias: found shape torch.Size([150]) in the checkpoint and torch.Size([5]) in the model instantiated
- decode_head.classifier.weight: found shape torch.Size([150, 256, 1, 1]) in the checkpoint and torch.Size([5, 256, 1, 1]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/inside-tech/Desktop/image_segmentation/.venv/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
Epoch 1/50 [Train]:   0%|          | 0/57 [00:00<?, ?it/s, loss=1.71, IoU=0, Dice=0]

outputs.shape: torch.Size([8, 5, 64, 64]), masks.shape: torch.Size([8, 64, 64])
outputs.dtype: torch.float32, masks.dtype: torch.int64


Epoch 1/50 [Train]:   5%|▌         | 3/57 [00:01<00:28,  1.90it/s, loss=1.68, IoU=0, Dice=0]